# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    exp = 0
    invested = P['Amount Invested'].sum()
    for i in range(len(P)):
        weight = P.at[i, 'Amount Invested'] / invested
        exp += (rf + P.at[i, 'Beta'] * mkt_rp) * weight
    
    return exp

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [4]:
def volatility(P, cor):
    invested = P['Amount Invested'].sum()
    var = 0
    for i in range(len(P)):
        weight = P.at[i, 'Amount Invested'] / invested
        var += (weight**2) * (P.at[i, 'Volatility'] ** 2)
    for i in range(len(P) - 1):
        w1 = P.at[i, 'Amount Invested'] / invested
        newvar = P.at[i, 'Volatility']
        for j in range (i + 1, len(P)):
            w2 = P.at[j, 'Amount Invested'] / invested
            newvar2 = P.at[j, 'Volatility']
            var += 2 * w1 * w2 * newvar * newvar2 * cor
    return var ** (1/2)

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [5]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    risky_asset = vol / mkt_vol
    exp = rf + risky_asset * mkt_rp
    return exp

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [6]:
def CAPM_beta(return_path, factors_path):
    #Write your code here
    rdf = pd.read_csv(return_path)
    fdf = pd.read_csv(factors_path)
    
    fdf['year'] = fdf['date'] // 100
    fdf['month'] = fdf['date'] % 100
    
    merged_df = pd.merge(rdf, fdf[['year', 'month', 'Mkt-RF', 'RF']], on=['year', 'month'], how='inner')
    merged_df['Excess_Return'] = merged_df['RET'] - merged_df['RF'] / 100  
    merged_df['Excess_Market_Return'] = merged_df['Mkt-RF'] / 100
    
    cov_df = merged_df.groupby('CUSIP').apply(lambda x: pd.Series({
        'BETA': np.cov(x['Excess_Return'], x['Excess_Market_Return'])[0, 1] / x['Excess_Market_Return'].var(),
        'RET': x['RET'].mean()
    }))
    
    cov_df.rename(columns={'RET': 'Expected_Return'}, inplace=True)
    rdf = cov_df[['Expected_Return', 'BETA']].reset_index()
    
    output = pd.DataFrame(data={
        "CUSIP": rdf['CUSIP'],
        "EXP_RETURN": rdf['Expected_Return'],
        "BETA": rdf['BETA'],
    })
    
    return output

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [7]:
def slope(data):
    coefficients = np.polyfit(data['BETA'], data['EXP_RETURN'], deg=1)
    sml = coefficients[1]
    return sml